In [ ]:
# Install Libraries
%%capture
import warnings
warnings.filterwarnings("ignore")
# All LangChain libraries for implementing logic chaining
%pip install -U langchain
%pip install -U langchain_community
%pip install -U langchain-huggingface
%pip install -U langchain_experimental
%pip install -U langchain_openai

%pip install -U unstructured
%pip install -U sentence-transformers

%pip install -U Neo4jGraph
%pip install -U py2neo
%pip install -U spacy
%pip install -U rdflib-neo4j
# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

from spacy import load, displacy
import pandas as pd
from google.colab import userdata
import string

In [ ]:
# Load from colab note
NEO4J_USERNAME = "neo4j"
NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
HF_API_KEY = userdata.get('HF_API_KEY')

# Set up connection to graph instance using LangChain
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)



In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

# define huggingface generation endpoint
hf_llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3", # Model Name
    task="text-generation",                       # task as generating a text response
    max_new_tokens=150,                           # maximum numbers of generated tokens
    do_sample=False,                              # disables sampling
    huggingfacehub_api_token=HF_API_KEY           # 🤗 huggingface API token
)

##### Caution: Run, only if you wish to delete the content of database

In [ ]:
# clean the neo4j dataset
## All nodes and relationships.
kg.query("MATCH (n) DETACH DELETE n")
## All indexes and constraints.
kg.query("CALL apoc.schema.assert({},{},true) YIELD label, key RETURN *")

# check if the dataset empty
kg.query("MATCH (n) RETURN count(n)")

[{'count(n)': 0}]

In [ ]:
import json

allowed_dependencies = {
     'acomp','advmod','agent','amod','attr','aux','auxpass',
     'case','cc','ccomp','compound','conj','det','dobj',
     'nmod','nsubj','nsubjpass',
     'pcomp','pobj','prep','poss','ROOT','xcomp'
}

def has_required_dependencies(doc, allowed_dependencies):
    if not {token.dep_ for token in doc}.issubset(allowed_dependencies):
        return False

    return ("is a" in doc.text.lower() or "is an" in doc.text.lower()) or \
            (any(token.dep_ == 'ROOT' for token in doc) and \
            any(token.dep_ in {'nsubj', 'nsubjpass'} for token in doc) and \
            any(token.dep_ in {'dobj', 'pobj'} for token in doc))

nlp = load("en_core_web_sm")
unhandled_sentences=set()
all_graphs = []
with open("DSA_knowledge.txt", "r") as file:
    sentences = file.read()

for sentence in [s.strip().rstrip(string.punctuation) for s in sentences.strip().split('\n') if s.strip()]:
  doc = nlp(sentence)
  # displacy.render(doc, style="dep", jupyter=True, options={'distance': 90})
  if not has_required_dependencies(doc, allowed_dependencies):
      unhandled_sentences.add(sentence)
      continue

  try:
    temp_graph = {
        "nodes": {},  # {'nodes': {0: {'pos': 0, 'label': 'X', 'dep': 'nsubj'}, 4: {'pos': 4, 'label': 'Y', 'dep': 'pobj'}},
        "edges": [],  # 'edges': [(0, 4, 'is subclass of')]}
        "sentence": sentence
    }

    edge_mapping = {
        'subject_nodes': {},  # {1: {0}} # multiple subject nodes possible
        'object_nodes': {},   # {1: 4}
        'edge_ids': set()     # {1}
    }

    temp_graph["nodes"] = {token['id']: {"pos": token['id'], "label": doc.text, "dep": token['dep']}
                          for token, doc in zip(doc.to_json()['tokens'], doc)}

    temp_graph["edges"] = [(token['head'], token['id'], token['dep'])
                            for token in doc.to_json()['tokens'] if token['head'] != token['id']]

    root_node = list(filter(lambda node: temp_graph["nodes"][node]['dep'] == 'ROOT', temp_graph["nodes"]))[0]
    stopping = False
    while not stopping:
      for edge in sorted(temp_graph["edges"], key=lambda x: abs(x[0] - x[1])):

        source_pos, target_pos, meta = edge

        if source_pos not in temp_graph["nodes"] or target_pos not in temp_graph["nodes"]:
            continue
        #print(edge)
        source_metadata = temp_graph["nodes"][source_pos]
        target_metadata = temp_graph["nodes"][target_pos]
        try:
            match (source_metadata, meta, target_metadata):
                case {'label': s, **source}, 'compound' | 'amod' | 'aux' |'auxpass' | 'advmod', {'label': t, **target}:
                    source_metadata['label'] = f"{t} {s}"
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'agent', {'label': t, **target}:
                    source_metadata['label'] = f"{s} {t}"
                    next_node = next((n for src, n, label in temp_graph["edges"] if src == target_pos and label == 'pobj'), None)
                    edge_mapping['edge_ids'].add(source_pos)
                    edge_mapping['object_nodes'][source_pos] = next_node
                    temp_graph['edges'].append((source_pos, next_node, 'pobj'))
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == target_pos and edge[1] == next_node), temp_graph['edges']))
                    del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'case' | 'cc', {'label': t, **target}:
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'det', {'label': t, **target}:
                    temp_graph['nodes'][source_pos]['det'] = t
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'attr'|'acomp', {'label': 'subclass'|'attribute'|'dimension'|'kind'|'threat'|'result'|'type'|'equal'|'form', **target}: #is(head)--attr--subclass(tail)--prep--of(child)--pobj--Risk
                    next_node = next((n for src, n, label in temp_graph["edges"] if src == target_pos and label == 'prep'), None)
                    obj_node = next((n for src, n, label in temp_graph["edges"] if src == next_node and label == 'pobj'), None)
                    source_metadata['label'] = f"{s} {target_metadata['label']} {temp_graph['nodes'][next_node]['label']}" #is-->issubclassof
                    edge_mapping['edge_ids'].add(source_pos)
                    edge_mapping['object_nodes'][source_pos] = obj_node
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges'])) # remove edge: is--subclass
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == target_pos and edge[1] == next_node), temp_graph['edges'])) # remove edge: subclass--of
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == next_node and edge[1] == obj_node), temp_graph['edges'])) # remove edge: of--Y
                    temp_graph['edges'].append((source_pos, obj_node, 'pobj')) #connect edge from 'is' node to obj node
                    del temp_graph['nodes'][target_pos] # remove node: 'subclass'
                    del temp_graph['nodes'][next_node]  # remove node: 'of'
                    continue

                case {'label': s, **source}, 'attr'|'acomp', {'label': t, **target}: #is-attr-Y
                    edge_mapping['edge_ids'].add(source_pos)
                    edge_mapping['object_nodes'][source_pos] = target_pos
                    continue

                case {'dep': 'ROOT', 'label': s, **source}, 'prep'|'xcomp', {'label': t, **target}: #attributes(ROOT)--prep--to #helps--xcomp--see--pobj--X
                    source_metadata['label'] = f"{s} {t}"
                    next_node = next((n for src, n, label in temp_graph["edges"] if src == target_pos and label in {'pobj', 'dobj'}), None)
                    if next_node:
                      temp_graph['edges'].append((source_pos, next_node, 'pobj'))
                      temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                      temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == target_pos and edge[1] == next_node), temp_graph['edges']))
                      del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'prep', {'label': t, **target}: #*-dobj-assessment--prep--of|*-attr-(a)dimension-prep-of
                    if next((n for src, n, label in temp_graph["edges"] if n == source_pos and label == 'attr'), None) is None:
                      next_node = next((n for src, n, label in temp_graph["edges"] if src == target_pos and label in {'pobj'}), None)
                      if next_node: #Date-prep-of-pobj-birth
                        source_metadata['label'] = f"{s} {t} {temp_graph['nodes'][next_node]['label']}"
                        temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                        temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == target_pos and edge[1] == next_node), temp_graph['edges']))
                        del temp_graph['nodes'][target_pos]
                        del temp_graph['nodes'][next_node]
                      else:
                        edge_mapping['edge_ids'].add(target_pos)
                        edge_mapping['subject_nodes'].setdefault(target_pos, set()).add(source_pos)
                        temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    continue

                case {'label': s, **source}, 'poss', {'label': t, **target}:
                    next_node = next((n for src, n, label in temp_graph["edges"] if src == source_pos and label == 'conj'), None)
                    if next_node:
                      temp_graph['edges'].append((temp_graph["nodes"][next_node]['label'],
                                                  temp_graph["nodes"][target_pos]['label'],
                                                  'of'))
                    continue

                case {'label': s, **source}, 'nmod', {'label': t, **target}:
                    source_metadata['label'] = f"{t} {s}"
                    incoming_node = next((src for src, n, label in temp_graph["edges"] if target == source_pos and label == 'nsubj'), None)
                    if 'conj' in target:
                        target['conj']['nodeId'] = target['conj']['text'] + f" {s}"
                        edge_mapping['subject_nodes'][incoming_node].add(target['conj']['nodeId'])
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'conj', {'label': t, **target}:
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    temp_graph['nodes'][source_pos]['conj'] = {'text': t, 'nodeId': target_pos}
                    continue

                case {'label': s, **source}, 'pcomp', {'label': t, **target}: #in--pcomp--explaining--dobj--x
                    temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                    next_node = next((n for src, n, label in temp_graph["edges"] if src == target_pos and label in {'pobj', 'dobj'}), None)
                    if next_node:
                      temp_graph['nodes'][root_node]['label'] += f" {t}"  #if not work f" {temp_graph['nodes'][root_node]['label']} {t}
                      temp_graph['edges'].append((root_node, next_node, 'dobj'))
                      temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == target_pos and edge[1] == next_node), temp_graph['edges']))
                      del temp_graph['nodes'][source_pos]
                      del temp_graph['nodes'][target_pos]
                    continue

                case {'label': s, **source}, 'ccomp', {'label': t, **target}: #Design interface can help users understand AI decisions
                    next_node = next((n for src, n, label in temp_graph["edges"] if src == target_pos and label == 'nsubj'), None)
                    if next_node:
                      edge_mapping['object_nodes'][source_pos] = next_node
                    continue

                case {'label': s, **source}, 'nsubj' | 'nsubjpass', {'label': t, **target}:
                    edge_mapping['edge_ids'].add(source_pos)
                    edge_mapping['subject_nodes'].setdefault(source_pos, set()).add(target_pos)
                    if 'conj' in target:
                      edge_mapping['subject_nodes'][source_pos].add(target_metadata['nodeId'])
                    continue

                case {'label': s, **source}, 'dobj' | 'pobj', {'label': t, **target}:
                    if next((src for src, n, label in temp_graph["edges"] if src == source_pos and label == 'prep'), None):
                      source_metadata['label'] = f"{s} {t}"
                      temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == source_pos and edge[1] == target_pos), temp_graph['edges']))
                      del temp_graph['nodes'][target_pos]
                    #assign object outside loop
                    continue

                case another:
                    print("another:", edge)
                    unhandled_sentences.add(sentence)
                    stopping = True
                    continue

        except Exception as e:
                print(f"Error occurred in sentence: {sentence} with edge: {edge}, error: {e}")
                unhandled_sentences.add(sentence) # throw error
                stopping = True
                continue
      else:
          break

    # Update object nodes
    edge_mapping['object_nodes'].update({
        edge_id: next((tail for head, tail, meta in temp_graph['edges']
                      if meta in {'dobj', 'pobj'}), None)
        for edge_id in edge_mapping['edge_ids']
        if edge_id not in edge_mapping['object_nodes']
    })

    for edge_id, obj_node in edge_mapping['object_nodes'].items():
        if obj_node is None:
            print(f"Missing object node for edge ID: {edge_id}")

    # create final mapping
    for edge_id in edge_mapping['edge_ids']:
        subject_nodes = edge_mapping['subject_nodes'][edge_id]
        object_node = edge_mapping['object_nodes'][edge_id]
        edge_node = temp_graph['nodes'][edge_id]
        for subject_node in subject_nodes:
            #temp_graph['edges'].append((subject_node, object_node, edge_node['label']))
            temp_graph['edges'].append((temp_graph["nodes"][subject_node]['label'],
                                        temp_graph["nodes"][object_node]['label'],
                                        edge_node['label']))
            temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == edge_id and edge[1] == subject_node), temp_graph['edges']))

        temp_graph['edges'] = list(filter(lambda edge: not (edge[0] == edge_id and edge[1] == object_node), temp_graph['edges']))
        del temp_graph['nodes'][edge_id]

    temp_graph['edges'] = list(set(temp_graph['edges'])-set([edge for edge in temp_graph['edges'] if edge[2] in allowed_dependencies]))

    #all_graphs.append(temp_graph)
    all_graphs.append({
      "edges": temp_graph["edges"],
      "sentence": temp_graph["sentence"]
    })
    for source, target, edge in temp_graph['edges']:
        kg.query(
          """
          CALL apoc.merge.node([$source_label], {label:$source_label}) YIELD node AS s
          CALL apoc.merge.node([$target_label], {label:$target_label}) YIELD node AS t
          CALL apoc.merge.relationship(s, $edge_label, {sentence: $sentence}, {}, t, {}) YIELD rel
          RETURN s,t,rel
          """
          , params={
          'source_label': temp_graph['nodes'][source]['label'].replace(" ", "_"),
          'target_label': temp_graph['nodes'][target]['label'].replace(" ", "_"),
          'edge_label': edge.lower().replace(" ", "_"),
          'sentence': sentence
        })

  except Exception as e:
    print(f"Failed to process sentence: {sentence}, error: {e}")
    unhandled_sentences.add(sentence)


with open("unhandled_sentences.txt", "w") as file:
  for unhandled in unhandled_sentences:
    file.write(unhandled + "\n")

# Training data - for fine tuning the HF model:
# Structure of json file is to be updated(current json edge has position of connected node, which we will change to node name)
with open('graph_data.json', 'w') as json_file:
    json.dump(all_graphs, json_file, indent=4, ensure_ascii=False)

NameError: name 'load' is not defined

In [ ]:
import json
# load JSON file
def load_json(path):
  with open(path, 'r') as file:
    return json.load(file)

import textwrap
# Prints the text with lines wrapped to a maximum width of 80 characters
def clean_print(text):
    return print(textwrap.fill(text, width=80))

In [ ]:

CYPHER_GENERATION_TEMPLATE = """Task:
Generate Cypher statement to query a Neo4j graph database.

Instructions:
* Only use the provided relationship types, node labels, and properties in the schema.
* Do not use any other relationship types, properties, or node labels that are not provided.
* Always follow the correct relationship direction.
* Ensure that the query follows the correct Cypher syntax.

Schema:
{schema}

Examples:
Here are a few examples of generated Cypher statements for particular questions:

# What is Bias?
    MATCH (s:Bias)-[r]-(t)
    RETURN s,r,t

# What might introduce Bias?
    MATCH (s)-[r:might_introduce]->(t:Bias)
    RETURN s,r,t

The question is:
{question}

The generated Cypher statement:"""

# Define schema
schema = """Node: Risk
Properties: label
Relationships: (Bias)-[:is]->(Risk)
"""

In [ ]:
# build the query prompt template
from langchain.prompts.prompt import PromptTemplate
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template=CYPHER_GENERATION_TEMPLATE
)

# initilize the chain
from langchain.chains import GraphCypherQAChain
cypherChain = GraphCypherQAChain.from_llm(
    graph=kg,
    llm=hf_llm,
    #cypher_llm=hf_llm,                    # see intermediate steps
    cypher_prompt=CYPHER_GENERATION_PROMPT,   # cypher generation prompt
    verbose=True,
    allow_dangerous_requests=True
)


In [ ]:
# test the chain
question = "What is Requirement?"
res = cypherChain.run(question)
clean_print('Question: '+question)
clean_print('Response: '+res)



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (s:Requirement)-[r]-(t)
RETURN s,r,t

Full Context:
[{'s': {'label': 'Requirement'}, 'r': ({'label': 'Fairness'}, 'is', {'label': 'Requirement'}), 't': {'label': 'Fairness'}}, {'s': {'label': 'Requirement'}, 'r': ({'label': 'Explainability'}, 'is', {'label': 'Requirement'}), 't': {'label': 'Explainability'}}]

> Finished chain.
Question: What is Requirement?
Response:  Fairness and Explainability are Requirements.


In [ ]:
question = "What is Bias?"
res = cypherChain.run(question)
clean_print('Question: '+question)
clean_print('Response: '+res)



> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (s:Bias)-[r]-(t)
RETURN s,r,t

Full Context:
[{'s': {'label': 'Bias'}, 'r': ({'label': 'Bias'}, 'is', {'label': 'Risk'}), 't': {'label': 'Risk'}}, {'s': {'label': 'Bias'}, 'r': ({'label': 'Algorithmic_Bias'}, 'is_subclass_of', {'label': 'Bias'}), 't': {'label': 'Algorithmic_Bias'}}, {'s': {'label': 'Bias'}, 'r': ({'label': 'Historical_Bias'}, 'is_subclass_of', {'label': 'Bias'}), 't': {'label': 'Historical_Bias'}}, {'s': {'label': 'Bias'}, 'r': ({'label': 'TrainTestSplit'}, 'might_introduce', {'label': 'Bias'}), 't': {'label': 'TrainTestSplit'}}, {'s': {'label': 'Bias'}, 'r': ({'label': 'Bias'}, 'is_threat_to', {'label': 'Fairness'}), 't': {'label': 'Fairness'}}]

> Finished chain.
Question: What is Bias?
Response:  Bias is a type of risk. It can be algorithmic bias, historical bias,
or it can be introduced by the train-test split. Bias is a threat to fairness.
